In [ ]:
g_zone=
  function (ref, test, prevalence = NULL, criterion.values = c(lrn, 
                                                               lrp), return.all = F) 
  {
    find.closest <- function(M, crit) {
      M[which(is.na(M))] = Inf
      mindiff = min(abs(M - crit))
      which((M == crit + mindiff) | (M == crit - mindiff), 
            arr.ind = T)
    }
    df = data.frame(cbind(ref, test))
    if (is.null(prevalence)) 
      prevalence = sum(df$ref == 1)/length(df$ref)
    tab = as.matrix(table(df$test, df$ref))#
    totpos = sum(tab[, 2])
    totneg = sum(tab[, 1])
    rdf = data.frame(thresholds = unique(sort(test)),
                     d0 = tab[,"0"], d1 = tab[, "1"], row.names = 1:nrow(tab))
    rdf$tot = rowSums(tab)
    rdf$TP = unname(rev(cumsum(rev(tab[, 2]))))
    rdf$FP = unname(rev(cumsum(rev(tab[, 1]))))
    rdf$TN = totneg - rdf$FP
    rdf$FN = totpos - rdf$TP
    rdf$tpr = rdf$TP/totpos
    rdf$fpr = rdf$FP/totneg
    rdf$preodds = prevalence/(1 - prevalence)
    rdf$sp = 1 - rdf$fpr
    rdf$se = rdf$tpr
    rdf$neg.lr = (1 - rdf$se)/rdf$sp
    rdf$pos.lr = rdf$se/(1 - rdf$sp)
    rdf$negpostodds = rdf$neg.lr * rdf$preodds
    rdf$pospostodds = rdf$pos.lr * rdf$preodds
    rdf$negpostprob = rdf$negpostodds/(rdf$negpostodds + 1)
    rdf$pospostprob = rdf$pospostodds/(rdf$pospostodds + 1)
    ltpos = find.closest(rdf$neg.lr, criterion.values[1])
    utpos = find.closest(rdf$pos.lr, criterion.values[2])
    rdf$threshold[ltpos]
    rdf$threshold[utpos]
    rdf$threshold[which(rdf$neg.lr <= criterion.values[1])]
    rdf$threshold[which(rdf$pos.lr >= criterion.values[2])]
    if (length(ltpos) > 1 | length(utpos) > 1) 
      warning("Multiple thresholds found.")
    if (return.all) {
      return(list(table = rdf, thresholds = c(lt = as.numeric(rownames(tab))[ltpos], 
                                              ut = as.numeric(rownames(tab))[utpos])))
    }
    else return(c(lt = as.numeric(rownames(tab))[ltpos], ut = as.numeric(rownames(tab))[utpos]))
  }



